# Установка Hadoop


## Docker compose

In [ ]:
version: '3' # Версия docker-compose
services:
# namenode управляет файловой системой и обеспечивает правильную работу кластера, распределяя задачи между DataNode и поддерживая целостность данных
  namenode:
    image: bde2020/hadoop-namenode:2.0.0-hadoop3.2.1-java8 # Образ для namenode
    container_name: namenode # имя контейнера
    hostname: namenode # имя хоста внутри сети Докера, по этому хосту могут обращаться другие контейнеры
    environment:
      - CLUSTER_NAME=test # название кластера
      - CORE_CONF_fs_defaultFS=hdfs://namenode:8020 # ссылка на файловую систему hdfs
      - HDFS_CONF_dfs_replication=1 # количество реплик для данных в  hdfs
      - HDFS_CONF_dfs_permissions_enabled=false # Проверка разрешений
    ports:
      - "9870:9870" # порт для веб-интерфейса name node
      - "9000:9000" # RPC порт для взаимодействия
    volumes:
      - namenode:/hadoop/dfs/name # Привязывает том namenode к директории внутри контейнера, что позволяет сохранять метаданные файловой системы.
    networks:
      - hadoop # название сети к которой подключается контейнер
# datanode обеспечивает хранение и доступ к данным
  datanode:
    image: bde2020/hadoop-datanode:2.0.0-hadoop3.2.1-java8 # Образ для datanode
    container_name: datanode # Название контейнера
    environment:
      - CORE_CONF_fs_defaultFS=hdfs://namenode:8020 # указываем как обратиться к namenode
      - HDFS_CONF_dfs_datanode_hostname=datanode # имя хоста datanode
    volumes:
      - datanode:/hadoop/dfs/data # привязываем datanode к директории
    ports:
      - "9864:9864" # порт для коммуникации
    networks:
      - hadoop # Название сети
    depends_on:
      - namenode # указываем что этот контейнер должен быть запущен после запуска namenode
#  recourcemanager является  координатором который распределяет ресурсы и контролирует выполнение приложений
  resourcemanager:
    image: bde2020/hadoop-resourcemanager:2.0.0-hadoop3.2.1-java8 # образ для recourcemanager
    container_name: resourcemanager # название контейнера
    environment:
      - YARN_CONF_yarn_resourcemanager_hostname=resourcemanager # имя хоста
      - YARN_CONF_yarn_scheduler_capacity_root_default_maximum_am_resource_percent=0.5 # максимальный процент ресурсов
    ports:
      - "8088:8088" # порт для веб
    networks:
      - hadoop
    depends_on:
      - namenode
      - datanode
# nodemanager управляет ресурсами на каждом отдельном узле и отвечает за выполнение задач, назначенных этому узлу
  nodemanager:
    image: bde2020/hadoop-nodemanager:2.0.0-hadoop3.2.1-java8
    container_name: nodemanager
    environment:
      - YARN_CONF_yarn_nodemanager_resource_memory_mb=4096 # объем оперативной памяти доступный для контенеров
      - YARN_CONF_yarn_nodemanager_resource_cpu_vcores=4 # количество виртуальных процессоров
    networks:
      - hadoop
    depends_on:
      - resourcemanager
# historyserver нужен для того, чтобы собирать, хранить и предоставлять доступ к информации о завершённых задачах и приложениях
  historyserver:
    image: bde2020/hadoop-historyserver:2.0.0-hadoop3.2.1-java8
    container_name: historyserver
    environment:
      - HADOOP_MAPRED_HOME=/hadoop # путь к домашнему коталогу
      - YARN_CONF_yarn_log_server_url=http://historyserver:8188/applicationhistory # ссылка для доступа к журналам
    ports:
      - "8188:8188"
    networks:
      - hadoop
    depends_on:
      - namenode
      - resourcemanager
      - nodemanager

networks:
  hadoop:
    driver: bridge

volumes: # создание томов для сохранения данных вне контейнеров
  namenode:
  datanode:

In [ ]:
datanode:
    image: bde2020/hadoop-datanode:2.0.0-hadoop3.2.1-java8
    environment:
      - CORE_CONF_fs_defaultFS=hdfs://namenode:8020
      - HDFS_CONF_dfs_datanode_hostname=datanode
    volumes:
      - datanode:/hadoop/dfs/data
    ports:
      - "9864:9864"
    networks:
      - hadoop
    depends_on:
      - namenode
    deploy:
      replicas: 3  # количество datanode контейнеров

## Конфигурационные файлы

* /etc/hadoop/core-site.xml CORE_CONF
* /etc/hadoop/hdfs-site.xml HDFS_CONF
* /etc/hadoop/yarn-site.xml YARN_CONF
* /etc/hadoop/httpfs-site.xml HTTPFS_CONF
* /etc/hadoop/kms-site.xml KMS_CONF
* /etc/hadoop/mapred-site.xml MAPRED_CONF

### core-site.xml


In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property><name>fs.defaultFS</name><value>hdfs://namenode:8020</value></property># указывает на URI по умолчанию для файловой системы, с которой будет работать Hadoop.
</configuration>


Возможные параметры:

* hadoop.tmp.dir: путь к временным директориям Hadoop. Это место для хранения временных данных и промежуточных результатов.

* io.file.buffer.size: размер буфера, используемого для операций ввода/вывода.

* fs.trash.interval: Интервал времени в минутах для хранения удалённых файлов в корзине перед окончательным удалением.

### hdfs-site.xml


In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>

<property><name>dfs.replication</name><value>1</value></property> # задает количество репликации для файлов, хранящихся в HDFS
<property><name>dfs.permissions.enabled</name><value>false</value></property> # определяет, включены ли проверки прав доступа на уровне HDFS
<property><name>dfs.namenode.name.dir</name><value>file:///hadoop/dfs/name</value></property> # определяет путь к локальной файловой системе, где namenode будет сохранять свои метаданные
<property><name>dfs.namenode.rpc-bind-host</name><value>0.0.0.0</value></property> # Задает IP-адрес, к которому namenode будет привязывать свой RPC
<property><name>dfs.namenode.servicerpc-bind-host</name><value>0.0.0.0</value></property> # IP-адрес для привязки сервиса RPC для namenode, который обслуживает запросы от datanode
<property><name>dfs.namenode.http-bind-host</name><value>0.0.0.0</value></property> # IP-адрес для HTTP-интерфейса namenode, который используется для доступа к веб-интерфейсу
<property><name>dfs.namenode.https-bind-host</name><value>0.0.0.0</value></property> # IP-адрес для HTTPS-интерфейса namenode, если включено безопасное соединение
<property><name>dfs.client.use.datanode.hostname</name><value>true</value></property> # Указывает, должен ли клиент HDFS использовать имена хостов datanode вместо IP-адресов для связи
<property><name>dfs.datanode.use.datanode.hostname</name><value>true</value></property> # Указывает, должен ли datanode сообщать namenode свое имя хоста вместо IP-адреса
</configuration>


Возможные параметры:
* dfs.blocksize: Размер блоков в HDFS.
* dfs.namenode.replication.min: Минимальное количество реплик для блока данных.
* dfs.datanode.data.dir: Путь к директории на datanode, где хранятся данные.


### yarn-site.xml

In [ ]:
<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>

<!-- Site specific YARN configuration properties -->

<property><name>yarn.resourcemanager.bind-host</name><value>0.0.0.0</value></property> # IP-адрес, к которому будет привязан recourcemanager
<property><name>yarn.nodemanager.bind-host</name><value>0.0.0.0</value></property> # IP-адрес, к которому nodemanager будет привязывать свой интерфейс
<property><name>yarn.nodemanager.bind-host</name><value>0.0.0.0</value></property> #
<property><name>yarn.timeline-service.bind-host</name><value>0.0.0.0</value></property> # IP-адрес для привязки Timeline Service, который собирает и предоставляет информацию о выполнении приложений в YARN
</configuration>


Возможные параметры:
* yarn.resourcemanager.hostname: Определяет hostname или IP-адрес recourcemanager.
* yarn.nodemanager.aux-services: Определяет дополнительные сервисы, которые будут запускаться на nodemanager.
* yarn.scheduler.maximum-allocation-mb: Максимальный объём памяти, который может быть выделен для одного контейнера.
* yarn.scheduler.minimum-allocation-mb: Минимальный объём памяти, который может быть выделен для одного контейнера.
* yarn.nodemanager.local-dirs: Путь к директориям на nodemanager, где будут храниться временные данные контейнеров.
* yarn.nodemanager.vmem-check-enabled: Включает или отключает проверку виртуальной памяти.



### mapred-site.xml

In [ ]:
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>

<property><name>yarn.nodemanager.bind-host</name><value>0.0.0.0</value></property> # IP-адрес, к которому будет привязан nodemanager
</configuration>


## Работа с hdfs

Основные команды:
* hdfs dfsadmin -report: проверка состояния файловой системы
* hdfs dfs -ls / : Отображение файлов и каталогов
* hdfs dfs -mkdir -p <путь и название директории> : создание каталогов
* hdfs namenode -format : Форматирование hdfs
* hdfs dfs -put <файл> <путь куда скопировать> : копирование файла в hdfs
* hdfs dfs -cat <путь до файла> : просмотр содержимого
* hdfs dfs -head <путь до файла> : просмотр первых нескольких строк
* hdfs dfs -tail <путь до файла> : просмотр последних нескольких строк
* hdfs dfs -cp <путь откуда> <путь куда> : копирование файла
* hdfs dfs -mv <путь откуда> <путь куда> : перемещение файла
* hdfs dfs -get <путь откуда hdfs> <путь куда локальный компьютер> : скачивание файла на локальный компьютер
* hdfs dfs -rm <путь до файла> : удаление файла
* hdfs dfs -rm -r <путь до каталога> : удаление каталога

